In [1]:
import pandas as pd
import numpy as np

import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('gfmt_sleep.csv', na_values='*')

# Add column for insomnia
df['insomnia'] = df['sci'] <= 16

df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence when incorrect hit,confidence when correct reject,confidence when incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True


In [3]:
# Options for x- and y- selector; omit part. num., gender, and insomnia
xy_options = list(
    df.columns[~df.columns.isin(["participant number", "gender", "insomnia"])]
)

x_selector = bokeh.models.Select(
    title="x", options=xy_options, value="percent correct", width=200,
)

y_selector = bokeh.models.Select(
    title="y", options=xy_options, value="confidence when correct", width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by", options=["none", "gender", "insomnia",], value="none", width=200,
)

In [4]:
render_cds = bokeh.models.ColumnDataSource(
    {
        "x": df["percent correct"],
        "y": df["confidence when correct"],
        "color": ["#1f77b3"] * len(df),
    }
)

In [5]:
cds = bokeh.models.ColumnDataSource(df)


In [7]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label="percent correct",
    y_axis_label="confidence when correct",
)

circle = p.circle(
    source=render_cds, x="x", y="y", fill_color="color", line_color="color",
)

In [9]:
jscode = """
// Extract what we want to color by from selector
var colorby = colorby_selector.value;

// View of the colors for convenience
var colors = render_cds.data['color'];

// Convenient to have the number of data points
var n = colors.length;

// New data
render_cds.data['x'] = cds.data[x_selector.value];
render_cds.data['y'] = cds.data[y_selector.value];

// Update colors
if (colorby === 'none') {
    for (var i = 0; i < n; i++) {
        colors[i] = '#1f77b3';
    }
}
else if (colorby === 'gender') {
    for (var i = 0; i < n; i++) {
        if (cds.data['gender'][i] === 'f') {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}
else if (colorby === 'insomnia') {
    for (var i = 0; i < n; i++) {
        if (cds.data['insomnia'][i]) {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}

// Update axis labels to reflect what was selected
xaxis[0].axis_label = x_selector.value;
yaxis[0].axis_label = y_selector.value;

render_cds.change.emit();
"""

args = dict(
    render_cds=render_cds,
    cds=cds,
    colorby_selector=colorby_selector,
    x_selector=x_selector,
    y_selector=y_selector,
    xaxis=p.xaxis,
    yaxis=p.yaxis,
)

In [10]:
colorby_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
x_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
y_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

In [11]:
layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

bokeh.io.show(layout)

In [12]:
bokeh.io.save(layout, 'sleep_facial_recognition.html')

/opt/miniconda3/envs/geoviews/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/miniconda3/envs/geoviews/lib/python3.9/site-packages/bokeh/io/saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/lianamerk/git/fermap/sleep_facial_recognition.html'